<a href="https://colab.research.google.com/github/Nyandala/To-Vaccinate-or-Not-to-Vaccinate-It-s-not-a-Question/blob/main/Vaccine_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load packages

In [1]:
import pandas as pd
import numpy as np

import string
import re
import spacy
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import TreebankWordTokenizer

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression,SGDClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# Load data

In [2]:
train = pd.read_csv('/content/drive/MyDrive/Data_Vaccinate or not/Train (2).csv')
test = pd.read_csv('/content/drive/MyDrive/Data_Vaccinate or not/Test (2).csv')

# Data preprocessing

In [3]:
display(train.head())
display(test.head())

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


,tweet_id,safe_text
0,00BHHHP1,<user> <user> ... &amp; 4 a vaccine given 2 he...
1,00UNMD0E,Students starting school without whooping coug...
2,01AXPTJF,"I'm kinda over every ep of <user> being ""rippe..."
3,01HOEQJW,How many innocent children die for lack of vac...
4,01JUKMAO,"CDC eyeing bird flu vaccine for humans, though..."


In [4]:
train.drop(train.loc[train['label']==0.666667].index, inplace=True)
train['label'].value_counts()

 0.000000    4908
 1.000000    4053
-1.000000    1038
 0.666667       1
Name: label, dtype: int64

In [5]:
display(train.isnull().sum())
display(test.isnull().sum())

tweet_id     0
safe_text    0
label        1
agreement    2
dtype: int64

tweet_id     0
safe_text    1
dtype: int64

In [6]:
train = train.dropna()
test = test.dropna()

In [7]:
display(train.isnull().sum())
display(test.isnull().sum())

tweet_id     0
safe_text    0
label        0
agreement    0
dtype: int64

tweet_id     0
safe_text    0
dtype: int64

In [8]:
# Replace URLs with 'url web'
pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'

train['safe_text'] = train['safe_text'].replace(to_replace = pattern_url, value = subs_url, regex = True)
test['safe_text'] = test['safe_text'].replace(to_replace = pattern_url, value = subs_url, regex = True)

In [9]:
# Contractions

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    
    return phrase

train['safe_text'] = train['safe_text'].apply(decontracted)
test['safe_text'] = test['safe_text'].apply(decontracted)

In [10]:
# Remove punctutaion

def remove_punctuation(safe_text):
  punc_numbers = string.punctuation + '0123456789'
  return ''.join([l for l in safe_text if l not in punc_numbers])
  
train['safe_text'] = train['safe_text'].apply(remove_punctuation)
test['safe_text'] = test['safe_text'].apply(remove_punctuation)

In [11]:
#Lowercase
train['safe_text'] = train['safe_text'].str.lower()
test['safe_text'] = test['safe_text'].str.lower()

# Feature extraction

In [12]:
#Tokenize
tk = TreebankWordTokenizer()

train['safe_text'] = train['safe_text'].apply(tk.tokenize)
test['safe_text'] = test['safe_text'].apply(tk.tokenize)

In [13]:
#Lemmatization

lemmatizer = WordNetLemmatizer()

def lemma(words, lemmatizer):
    return [lemmatizer.lemmatize(word) for word in words]

train['safe_text'] = train['safe_text'].apply(lemma, args=(lemmatizer, ))
test['safe_text'] = test['safe_text'].apply(lemma, args=(lemmatizer, ))

In [14]:
# Remove stop words
all_stopwords = stopwords.words('english')
all_stopwords.append(['user', 'url-web'])

def remove_stop_words(tokens): 
  all_stopwords = stopwords.words('english')
  new_sw =['user', 'url-web']
  all_stopwords.append(new_sw)   
  return [t for t in tokens if t not in all_stopwords]

train['safe_text'] = train['safe_text'].apply(remove_stop_words)
test['safe_text'] = test['safe_text'].apply(remove_stop_words)

In [15]:
train['safe_text'] = train.safe_text.apply(lambda x: '    '.join(x))
test['safe_text'] = test.safe_text.apply(lambda x: '    '.join(x))

In [16]:
# Select X and y

X = train['safe_text']
y = train['label']
X_test = test['safe_text']


In [17]:
#Split data

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state = 42)

In [18]:
#Vectoriser

tf = TfidfVectorizer( max_df = 0.3, ngram_range = (1, 2),min_df = 3)

X_train = list(X_train)
X_val = list(X_val)
X_test = list(X_test)

X_train = tf.fit_transform(X_train)
X_val = tf.transform(X_val)
X_test = tf.transform(X_test)

# Modelling

In [19]:
# Grid search

lg_gs = LogisticRegression(max_iter= 1000)
lg_gs_params = {'C':[0.1, 1, 10, 100]}
lg_gs_s = GridSearchCV(lg_gs, param_grid = lg_gs_params,scoring = 'recall_weighted')
lg_gs_s.fit(X_train, y_train)
y_pred_lg_gs_s = lg_gs_s.predict(X_val)

#Predict values based on new parameters
y_pred_lg_gs_test = lg_gs_s.predict(X_test)

# New Model Evaluation metrics 
print('Accuracy Score : ' + str(accuracy_score(y_val,y_pred_lg_gs_s)))
print('Precision Score : ' + str(precision_score(y_val,y_pred_lg_gs_s, average = None)))
print('Recall Score : ' + str(recall_score(y_val,y_pred_lg_gs_s, average = None)))
print('F1 Score : ' + str(f1_score(y_val,y_pred_lg_gs_s, average = None)))

Accuracy Score : 0.734
Precision Score : [0.69230769 0.76199616 0.7043673 ]
Recall Score : [0.1875     0.84199364 0.76988984]
F1 Score : [0.29508197 0.8        0.73567251]


# Submission file

In [ ]:
sub = pd.DataFrame({'tweet_id': test['tweet_id'], 'label': y_pred_lg_gs_test})
sub.head()

,tweet_id,label
0,00BHHHP1,1.0
1,00UNMD0E,1.0
2,01AXPTJF,0.0
3,01HOEQJW,1.0
4,01JUKMAO,0.0


In [ ]:
sub.to_csv('Vaccine_Hackathon_Nyandala.csv', index = False)